In [1]:
#Import Dependencies
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import time
import traceback
import copy
import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
path="redcap_levels/redcap_output/" #Path with BMI files from REDCap 
#Only those of interest - Taken from BMI_6-6-2023.csv

In [3]:
#Read excel files with data
high=pd.read_excel(os.getcwd()+'/BMI_exp_files'+"\\high_scans.xlsx") 
low=pd.read_excel(os.getcwd()+'/BMI_exp_files'+"\\low_scans.xlsx") 

In [4]:
#Needed for FN extraction
path_low=os.getcwd()+'/details_final/low'
path_high=os.getcwd()+'/details_final/high'

In [5]:
start=time.time()

In [6]:
BMI_data=pd.DataFrame() #Create empty dataframe

Changes compared to emphysema experiment:
- 'participant_id_only' to get only a 6-digit participant, without any errors from REDCap export
- Removed section with 'indices_delete'
- Changed 'split' conventions below, 'BMI' column added
- Removed '!!!'
- Keep volumes >300mm3
- Change to 'astype(str)' for TP
- Change to 'astype(float)' in convert function below
- Added 'all_values[2]!=2' condition in function below - Also added in emphysema, seems no problem
- 'Statistics - Not needed...', 'Resample' section, 'Keep only the largest nodule of each participant' paragraph etc. deleted
- Set 'plotting=False' by default to the function 'get_nodule_names_and_sizes'
- Sections 'only nodules <=100mm3' and below not copied since we will probably not need any pie plots
- We will only keep solid component
- Excluded cases with 10 findings by initial reading
- Excluded Peribronchial tissue

In [7]:
#Combine all dataframes with the different degrees of BMI

total_pat=0 #count number of participants

for file in os.listdir(path): #loop over dataframes
    print(file) #print BMI degree
    dataframe=file.split('.')[0] #Use degree of BMI as a name for each corresponding dataframe

    exec(dataframe+'=pd.read_csv(os.getcwd()+"/"+path+file)') #read each dataframe

    exec(dataframe+"['BMI']=str(dataframe.split('.')[0])") #Add column with BMI degree
    BMI_data=BMI_data.append(eval(dataframe)) #Combine each BMI dataframe to another dataframe
    total_pat=total_pat+len(eval(dataframe)) #Add number of participants of that degree to the total number of participants
    print(len(np.unique(eval(dataframe+"['participant_id_only']")))) #print num of unique participants of that degree of BMI

    assert(total_pat==len(BMI_data)) #Confirm that everything worked

high_redcap.csv
176
low_redcap.csv
176


In [8]:
BMI_data=BMI_data.reset_index(drop=True) #Reset indices

In [2]:
# Delete nodule in slice 72 (id 1 in syngo.via) due to being small 
# Find the row where 'column_to_match' matches the specific value
row_index = BMI_data[BMI_data['participant_id'] == 888119].index[0]

# Iterate over the columns and replace those containing 'n3' with NaN for the selected row
for col in BMI_data.columns:
    if 'n1' in col and 'n10' not in col and 'ai' not in col:
        try:
            BMI_data.at[row_index, col] = np.nan
        except:
            print('Column:',col, 'Value:',BMI_data.at[row_index, col])
            print(traceback.format_exc())

In [10]:
print("Number of unique participants is {}".format(len(np.unique(BMI_data['participant_id_only']))))
print("We have in total {} participants".format(len(BMI_data)))

Number of unique participants is 352
We have in total 352 participants


In [11]:
#BMI types ordered by severity - Taken from np.unique(BMI_data['BMI'])
severity=['high_redcap','low_redcap']

In [12]:
severity_nums=[1,0] #create integers from the most severe to the least one

In [13]:
severity_dict={} #create a dict with severity name and corresponding integer
for index,val in enumerate(severity):
    severity_dict[val]=severity_nums[index]
severity_dict

{'high_redcap': 1, 'low_redcap': 0}

In [14]:
BMI_data['Severity']=BMI_data['BMI'] #copy BMI column to severity column

In [15]:
BMI_data=BMI_data.replace({"Severity": severity_dict}) #replace severity with the numbered version of it

In [3]:
BMI_data=BMI_data.sort_values(by=['Severity'], ascending=False) #sort by severity of BMI
BMI_data=BMI_data.reset_index(drop=True) #Reset indices
BMI_data

## Inconsistencies
There are some participants for which the number of volumes is less than the number of nodules found. According to a radiologist this is because the volumes of ground-class nodules were not included in the downloads. 

In [17]:
for i in range(1,11): #Create empty columns to be filled with volumes of nodules below
    BMI_data['volume_all_n'+str(i)]=float(0)

In [1]:
# BMI_data.iloc[:,20:]

Kept only solid part of both solid and subsolid nodules, only if >=30mm3. These should also be changed in Excel file, as mentioned below

In [4]:
#Find cases in which we have both 'solid' and 'subsolid' volumes for same nodule keep only solid components if vol>30mm3
#Subsolid in redcap will be solid + the subsolid part. So only subsolid will be subsolid in redcap- solid in redcap.

vol_both_comps=[] #To be filled with participants who have both solid and subsolid component

for j in range(len(BMI_data)):
    
    for nod_num in range(1,11):

        # if BMI_data.loc[j,'participant_id'] in (low_pats+high_pats): #If participant in our experiment - Should define these above in the notebook

            solid=BMI_data.loc[j,'volume_solid_n'+str(nod_num)]
            subsolid=BMI_data.loc[j,'volume_subsolid_n'+str(nod_num)]
            
            if np.isnan(solid)==True and np.isnan(subsolid)==False: #If only subsolid component
                print("Only subsolid component for nod id",nod_num,'of participant',BMI_data.loc[j,'participant_id'],"Volume set to 0.0")
                BMI_data.loc[j,'volume_all_n'+str(nod_num)]=0.0 #These are taken into account in other file

            elif np.isnan(solid)==False and np.isnan(subsolid)==True: #If only solid component
                BMI_data.loc[j,'volume_all_n'+str(nod_num)]=solid
                if solid<30:
                     print("We should not have in REDCap a solid component <30mm3! Check again nod id",nod_num,'of participant',BMI_data.loc[j,'participant_id'])
                
            elif np.isnan(solid)==False and np.isnan(subsolid)==False: #If both solid and subsolid components

                # vol_both_comps.append(BMI_data.loc[j,'participant_id_only'])
        
                if solid>=30: #If solid component >30mm3
                    BMI_data.loc[j,'volume_all_n'+str(nod_num)]=solid
                else:
                    print("Both solid and subsolid components but solid <=30. Didn't keep any of them for nod id",nod_num,'of participant',BMI_data.loc[j,'participant_id'])
                
            else: 
                BMI_data.loc[j,'volume_all_n'+str(nod_num)]=np.nan #We only get in here if there is no nodule

In [20]:
# #Show these lists of participants with volumes for solid and subsolid components of the same nodule
# list(np.unique(vol_both_comps))

In [21]:
# #Show volumes for cases with solid and subsolid components
# check_cols=[col for col in BMI_data.columns if 'volume' in col]
# BMI_data.loc[BMI_data['participant_id'].isin(vol_both_comps)][check_cols]

In [5]:
# BMI_data = BMI_data[~BMI_data.participant_id.isin(vol_both_comps[1:])] #Remove these participants
BMI_data=BMI_data.reset_index(drop=True)
BMI_data

In [23]:
for name in severity_dict.keys(): #loop over names of ΒΜΙ degrees and print num of participants of each degree now
    print(name)
    #Transform each dataframe to have each participant 1 time
    exec(name+"=BMI_data[BMI_data['BMI']==name]") 
    
    print(len(BMI_data[BMI_data['BMI']==name]))

high_redcap
176
low_redcap
176


In [6]:
# high_redcap

#### Get information from REDCap for participants in the ΒΜΙ experiment - Use it below to get TP information

In [7]:
# BMI_data

In [26]:
#Patient IDs of individuals with low and high BMI
low_pats=[....]

high_pats=[....]
        
print("Number of individuals with low BMI is {}".format(len(low_pats)))
print("Number of individuals with high BMI is {}".format(len(high_pats)))

Number of individuals with low BMI is 176
Number of individuals with high BMI is 176


In [27]:
all_pats=high_pats+low_pats

In [8]:
TP_export=BMI_data[BMI_data['participant_id_only'].isin(all_pats)]
TP_export

### Get indices of TP for each participant - Use manually checked annotations

In [9]:
# high

In [32]:
#Replace errors of automation algorithm extraction
high=high.replace("!!!",'',regex=True) #Replace double exclamation marks 
low=low.replace("!!!",'',regex=True) #Replace double exclamation marks 

high=high.replace("xxx",'',regex=True) #Replace double exclamation marks 
low=low.replace("xxx",'',regex=True) #Replace double exclamation marks 

low=low.reset_index(drop=True) #Reset indices
high=high.reset_index(drop=True) #Reset indices

In [33]:
#Replace 0 with NaN - Output of algorithm is 0 but need nan below
high['0-100fn'].replace(0, np.nan, inplace=True)
high['100-300fn'].replace(0, np.nan, inplace=True)
high['300+ fn'].replace(0, np.nan, inplace=True)

low['0-100fn'].replace(0, np.nan, inplace=True)
low['100-300fn'].replace(0, np.nan, inplace=True)
low['300+ fn'].replace(0, np.nan, inplace=True)


high['0-100tp'].replace(0, np.nan, inplace=True)
high['100-300tp'].replace(0, np.nan, inplace=True)
high['300+ tp'].replace(0, np.nan, inplace=True)

low['0-100tp'].replace(0, np.nan, inplace=True)
low['100-300tp'].replace(0, np.nan, inplace=True)
low['300+ tp'].replace(0, np.nan, inplace=True)

In [10]:
# high

Cell below should be commented. It changes the number of TPs so that they are not the same as in the excel files by not considering TP with nodules measured by AI being <30mm3 (example 528787, 922165). The issue of not considering as FPs AI findings with vol <30mm3 is addressed elsewhere.  

In [35]:
# #This is to prevent considering vols <30 
# vol_cols=[col for col in low.columns if 'V' in col] #Get name of columns containing volumes of AI nodules

# BMI_deg=['low_fp','high_fp']

# for deg in BMI_deg: #Loop over emphysema degrees
#     for col in vol_cols: #Loop over columns with volumes
#         #If the volume is less than 30mm3 we should ignore them - set it along with the corresponding AI nod to '-'
#         for ind,val in eval(deg[:-3]+"[("+deg[:-3]+"['"+col+"']<=30) ]['"+col+"'].items()"):
#              #| ("+deg[:-3]+"['"+col+"']>300) - Not added since we might have a TP with vol>300 from AI which is <300 in REDCap
#             exec(deg[:-3]+"['"+col+"'].iloc[ind]=np.nan") #was '-' instead of nan
#             exec(deg[:-3]+"['AI_nod"+str(col[1:])+"'].iloc[ind]=np.nan")

In [36]:
#Select rows where we have at least one TP in any of the 0-100, 100-300, or 300mm3 volume subgroup
high_tp=high[(high['100-300tp'].notnull() | high['0-100tp'].notnull() | high['300+ tp'].notnull()) & high['participant_id'].notnull()]
low_tp=low[(low['100-300tp'].notnull() | low['0-100tp'].notnull() | low['300+ tp'].notnull()) & low['participant_id'].notnull()]

In [11]:
# low_tp

In [38]:
#Initialize empty dicts in the form {'pat_id1':[],'pat_id2':[],...}
high_dict=dict.fromkeys([str(numeric_string) for numeric_string in high_tp['participant_id'].values], [])
high_dict=[[key[:6],[]] for (key, value) in high_dict.items()] #Initialize list in the form [participant_id,[]]
high_dict = {item[0]: item[1] for item in high_dict} #convert to dictionary in the form {pat_1:[nod_1,...],...}
high_tp['participant_id']=list(high_dict.keys())

low_dict=dict.fromkeys([str(numeric_string) for numeric_string in low_tp['participant_id'].values], [])
low_dict=[[key[:6],[]] for (key, value) in low_dict.items()] #Initialize list in the form [participant_id,[]]
low_dict = {item[0]: item[1] for item in low_dict} #convert to dictionary in the form {pat_1:[nod_1,...],...}
low_tp['participant_id']=list(low_dict.keys())

In [39]:
AI_cols=[col for col in high_tp.columns if 'AI_nod' in col] #Get name of columns containing AI nodules

In [12]:
# high_tp

In [41]:
BMI_deg=['high_tp','low_tp'] #Initialize list with possible BMI degrees. 
#Could also be without the suffix '_tp' and then just use 'deg' instead of 'deg[:-3]' below - For now stays as is

for deg in BMI_deg: #Loop over BMI degrees
    print(deg)
    for ind_col,col in enumerate(AI_cols): #loop over AI columns with nodules and their ids
        
        #Following line to change nan with '-' since otherwise cannot check for string with 'L' below
        exec(deg[:-3]+"_tp['"+col+"']="+deg[:-3]+"_tp['"+col+"'].fillna('-')")

        exec(deg[:-3]+"_tp['"+str(col)+"'] = "+deg[:-3]+"_tp['"+str(col)+"'].astype(str)") #Type as string
        
        #Create a variable storing only those rows of df that a specific AI_nod col contains 'L' (denotes a TP)
        exec('temp='+deg[:-3]+'_tp['+deg[:-3]+"_tp['"+str(col)+"'].str.contains('L')]")
        
        if not temp.empty: #If we have TP for that participant

            for ind,pat in enumerate(temp['participant_id']): #Loop over all participants with TP in a specific AI col

                try: #To ensure that there are no errors
                    nod_id=temp.iloc[ind,ind_col+1][temp.iloc[ind,ind_col+1].find('L')+1:] #Get id
                    nod_id=nod_id.split(' ')[0] #To get actual id
                    exec(deg[:-3]+'_dict'+"['"+str(pat)+"'].append('"+nod_id+"')") #Add that to the dictionary
                except:
                    print(traceback.print_exc()) #print errors

high_tp
low_tp


In [13]:
# high_dict

In [43]:
for deg in BMI_deg: #Loop over BMI degrees
    for key,val in eval(deg[:-3]+'_dict.items()'): #Loop over all participants and keys
        #Replace keys like '04' with just the integer 4
        exec(deg[:-3]+"_dict['"+key+"']=[int(x) for x in "+deg[:-3]+"_dict['"+key+"']]")  

In [14]:
# high_dict

In [15]:
# low_dict

In [46]:
print("Number of TPs findings in high BMI:",len([y for x in high_dict.values() for y in x]),"Those come from",len(high_dict),"participants")
print("Number of TPs findings in low BMI:",len([y for x in low_dict.values() for y in x]),"Those come from",len(low_dict),"participants")

Number of TPs findings in high BMI: 67 Those come from 46 participants
Number of TPs findings in low BMI: 87 Those come from 59 participants


#### Extract TP information for the ids of TP nodules for each participant found above

In [47]:
cols_with_nod_ids=[col for col in TP_export.columns if 'nodule_id_n' in col] #Columns with nodule ids

In [48]:
#Keep only the participants of interest - Those in the low/high BMI lists
high_TP_fin=TP_export[TP_export['participant_id_only'].isin(high_pats)]
low_TP_fin=TP_export[TP_export['participant_id_only'].isin(low_pats)]

In [16]:
# high_TP_fin

In [50]:
#Convert participant ids to integers
high_dict= {int(k):v for k,v in high_dict.items()} 
low_dict= {int(k):v for k,v in low_dict.items()} 

In [51]:
#Keep in each df only those rows (participants) that have at least one TP nodule
high_TP_fin=high_TP_fin.loc[high_TP_fin['participant_id_only'].isin(list(high_dict.keys()))]
low_TP_fin=low_TP_fin.loc[low_TP_fin['participant_id_only'].isin(list(low_dict.keys()))]

In [17]:
# high_TP_fin

In [18]:
# low_TP_fin

In [54]:
#Until now we have the ids in the Nodule ID attribute on REDCap. We have to convert them to numbers from 1-10 that
#correspond to the REDCap attributes - initialize dict with {pat:empty} list pairs

true_ids_high_dict={key:[] for key,val in high_dict.items()} 
true_ids_low_dict={key:[] for key,val in low_dict.items()} 

In [55]:
def convert_to_true_ids(dictionary,df,cols_with_nod_ids,new_dictionary):
    
    #Get ids of where TP nodules stored in REDCap - from 1 to 10 - used to get information from respective columns below
    for pat,nods in dictionary.items(): #Loop over dictionary key-values
        true_nod_id=df.loc[df['participant_id_only'] == pat][cols_with_nod_ids].values[0].astype(float) #Get value of nodule 
        # 748658 nod id 1 is int and not float! This is why we added astype(float)

        for nod in nods: #Loop over nodules
            new_dictionary[pat].append(np.where(nod==true_nod_id)[0][0]+1) #Add true nod_id in dictionary
            
    return new_dictionary

In [56]:
#Apply function to get nodule id in REDCap from nodule value stored in it
true_ids_high_dict=convert_to_true_ids(high_dict,high_TP_fin,cols_with_nod_ids,true_ids_high_dict)
true_ids_low_dict=convert_to_true_ids(low_dict,low_TP_fin,cols_with_nod_ids,true_ids_low_dict)

In [57]:
print("Number of TPs ids in high BMI:",len([y for x in true_ids_high_dict.values() for y in x]),"Those come from",len(true_ids_high_dict),"participants")
print("Number of TPs ids in low BMI:",len([y for x in true_ids_low_dict.values() for y in x]),"Those come from",len(true_ids_low_dict),"participants")

Number of TPs ids in high BMI: 67 Those come from 46 participants
Number of TPs ids in low BMI: 87 Those come from 59 participants


In [19]:
# low_dict

## Add information from REDCap to lists

#### Explanation of REDCap values
- calcification - we keep 2-6 (calcified):

1. No=>1 
2. Fully calcified, centrally, popcorn, rum,other=>1-6

- pfn (numbers represent value in REDCap) - we keep 3 (atypical PFN/triangular lymph node) - caution: 2,3 exist also for typical/fissural (if for attachment_n7 value is 1). In general, both typical and atypical PFNs (intrapulmonary lymph nodes) are always non-calcified (calcification=1), have sharp borders and oval, triangular, or polygonal (rectangle or dumdbell rarely) shape -any of shape_==2 or 3 or 4- (also round but could be a misclassification of cancer). Also, most of them have a vascular, or fissure or pleural attachment (any of attachments 2,3,7 is 1). 
Typical PFNs always have a fissure attached (attach_7=1) while atypical may or may not be fissure attached:

1. No=>1
2. Typical PFN=>2
3. Atypical PFN/possible =>3
4. Don't know =>4

- attachment (if eg. attach_n1_2 is 0 means not pleural based) - we keep either n_2=1 (pleural) or n_4, n_5=1 (peribronchial/bronciovascular lymph node, if also central 'location', smooth 'edge', triangular or polygonal 'shape', and solid 'nodule_type'):

1. 0mm, reaching to pleura
2. pleural based 
3. juxtapleural
4. peribronchial
5. vascular attached
6. juxtavascular
7. fissural attached

- nodule type - we keep 3 (subsolid/ground glass)
1. solid
2. partial solid
3. pure ground glass

In [59]:
#Create cols to extract nodule information from REDCap
#For 'attachment_n' since only possible values are 0 or 1, we could also work with lists instead of dictionaries.
#More convenient dictionaries since we already have the pipeline for the rest

def extract_information(df):
    calcified_nodules=[col for col in df.columns if 'calcification_n' in col]
    pleural_nodules=[col for col in df.columns if 'attachment_n' in col and '___2' in col]
    subclass_ground_glass_nodules=[col for col in df.columns if 'nodule_type_n' in col] #value should be 3
    #for 'other' nodules no information - should check for each one of those what are their attributes - same for 'cancer'

    atypical_periphysural_fissural_pfn_triangular_lymph_nodes=[col for col in df.columns if 'pfn_n' in col] 
    attachment_to_distinguish_atypical_typical=[col for col in df.columns if 'attachment_n' in col and ('___7' in col)]
    
    peribronchial_bronchiovasc_ln=[col for col in df.columns if 'attachment_n' in col and ('___4' in col)]
    peribronchial_bronchiovasc_ln_2=[col for col in df.columns if 'attachment_n' in col and ('___5' in col)]
    
    location=[col for col in df.columns if 'loc_n' in col]
    shape=[col for col in df.columns if 'shape_n' in col]
    edge=[col for col in df.columns if 'edge_n' in col]
        
    #non-nodules do not exist (fibrosis/scar/pleural thickening and other (bone, tissue etc.))

    return (calcified_nodules, pleural_nodules, subclass_ground_glass_nodules,
            atypical_periphysural_fissural_pfn_triangular_lymph_nodes,attachment_to_distinguish_atypical_typical,
            peribronchial_bronchiovasc_ln,peribronchial_bronchiovasc_ln_2,location,shape,edge)

In [60]:
#Extract columns with nodules of interest for all dfs - same for low/high BMI categories
(calcified_nodules, pleural_nodules, subclass_ground_glass_nodules,
 atypical_periphysural_fissural_pfn_triangular_lymph_nodes,attachment_to_distinguish_atypical_typical,
 peribronchial_bronchiovasc_ln,peribronchial_bronchiovasc_ln_2,location,shape,edge)=extract_information(TP_export)

In [61]:
#Combine the above to a single list
all_cols=(calcified_nodules+pleural_nodules+subclass_ground_glass_nodules+
          atypical_periphysural_fissural_pfn_triangular_lymph_nodes+attachment_to_distinguish_atypical_typical+
          peribronchial_bronchiovasc_ln+peribronchial_bronchiovasc_ln_2+location+shape+edge)
#We get 60 columns in total

Keep track of nodule id and slice where it can be found

In [62]:
#Create dictionary with dictionaries in format: {pat:{nod_num:value},...}

def add_pat(dictionary,pat,nod_num,slices,vol_dict,volume): 
    
    if pat in dictionary: #If participant in dictionary add/replace one nod_id value to it. Same for volume
        dictionary[pat][nod_num]=slices
        vol_dict[pat][nod_num]=volume
        
    else: #If not, then initialize the value of that participant to an empty dictionary and fill in its values
        dictionary[pat]={}
        dictionary[pat][nod_num]=slices
        
        vol_dict[pat]={} #Same for dictionary with participant_ids and nod volumes
        vol_dict[pat][nod_num]=volume
    
    return dictionary,vol_dict

Peribronchial lymph nodes are included since only 'peribronchial tissue' should be excluded which shouldn't exist in REDCap

In [63]:
def TP_info_redcap(tp_or_fn='tp'):
    """Function to extract description of TP or FN nodules from REDCap."""

    # Make below variables accessible outside function without using 'return' and setting new variables:
    global calcified, calcified_vol, pleural, pleural_vol, sub_ground, sub_ground_vol, atypical_triangular, atypical_triangular_vol, per_fisu, per_fisu_vol, \
            peri_bronch, peri_bronch_vol, other_all, other_all_low, other_all_high, other_all_vol, atypical_triangular_low, per_fisu_low, peri_bronch_low, \
            calcified_low, pleural_low, sub_ground_low, atypical_triangular_high, per_fisu_high, peri_bronch_high, calcified_high, pleural_high, sub_ground_high

    #Initialize empty lists for nodule categories - also empty lists for their volumes to be filled in

    #For this and for the below one maybe also 'shape'=>3,4 means more benign. Not used for now
    atypical_triangular={} #'pfn_n'=>3 caution: 2,3 exist also for typical/fissural (if for attachement_n7 value is 1)
    atypical_triangular_vol={}
    per_fisu={} #'pfn_n'=>2,3 and attachment_n7 value is 1. Look above for more
    per_fisu_vol={}
    peri_bronch={} #'attachment'=> n4,n5=1
    peri_bronch_vol={}

    calcified={} #'calcification' =>2-6
    calcified_vol={}
    pleural={}#'attachment'=> n_2=1
    pleural_vol={}
    sub_ground={} #'nodule_type' =>3
    sub_ground_vol={}

    #For other type of nodules not in the above categories
    other_all={}
    other_all_low={}
    other_all_high={}
    other_all_vol={}

    #Non-nodule categories not exist in REDCap

    #For low/high BMI table - Not sure if needed given that we can obtain those from original dict combined with 
    #list of participants that belong to each degree, found above

    #Low BMI
    atypical_triangular_low={}
    per_fisu_low={}
    peri_bronch_low={}
    calcified_low={}
    pleural_low={}
    sub_ground_low={}

    #High BMI
    atypical_triangular_high={}
    per_fisu_high={}
    peri_bronch_high={}
    calcified_high={}
    pleural_high={}
    sub_ground_high={}


    if tp_or_fn=='tp':
        type_extract='true_ids_'
        attr='_TP_fin'
    else:
        type_extract='false_ids_'
        attr='_df_fn'


    #Implement above definition of each type of finding and store each variable in the corresponding dictionary

    for deg in BMI_deg: #Loop over low/high BMI degrees (in the form 'low_tp')
        
        #Loop over participant ids and actual nod_ids of TPs in REDCap - from 1 to 10
        for pat,nods in eval(type_extract+deg[:-3]+'_dict.items()'): 

            for nod_num in nods: #Loop over TP nodule ids
                
                flag=0 #Initialize a flag to help us distinguish atypical from fissural PFN - when both have 'pfn' value=3 
                flag_any=0 #Flag to check if we got in any of the below categories
                
                #Get columns with volume for a specific nodule id - need two statements to distinguish 'n1' from 'n10'
                if nod_num!=1:
                    nod_cols=[col for col in all_cols if '_n'+str(nod_num) in col]
                    vol_cols=[col for col in TP_export.columns if 'volume_all_n'+str(nod_num) in col]
                    slice_cols=[col for col in TP_export.columns if 'slice_number_n'+str(nod_num) in col]
                else:
                    nod_cols=[col for col in all_cols if '_n1' in col and '_n10' not in col]
                    vol_cols=[col for col in TP_export.columns if 'volume_all_n'+str(nod_num) in col and '_n10' not in col]
                    slice_cols=[col for col in TP_export.columns if 'slice_number_n'+str(nod_num) in col and '_n10' not in col]

                #  #For debug
    #             if pat==490144:
    #                 print(pat,nod_num)
    #                 print(eval(deg[:-3]+"_TP_fin["+deg[:-3]+"_TP_fin['participant_id']=="+str(pat)+"][nod_cols]"))
                    
                #Get all values and volumes for a specific nodule id. Order is as defined above:
                #calcified,pleural,subclass_ground_glass,atypical_periphysural_fissural,
                #attachment_to_distinguish_atypical_typical,peri_bronch_ln,peri_bronch_ln_2,location,shape,edge
                all_values=eval(deg[:-3]+attr+"["+deg[:-3]+attr+"['participant_id']=="+str(pat)+"][nod_cols].values[0]")
                all_volumes=eval(deg[:-3]+attr+"["+deg[:-3]+attr+"['participant_id']=="+str(pat)+"][vol_cols].values[0]")
                all_slices=eval(deg[:-3]+attr+"["+deg[:-3]+attr+"['participant_id']=="+str(pat)+"][slice_cols].values[0]")

                
                if np.isnan(all_values[0]): #'calcification'
                    print('For ', pat, 'we have nan calcified in nod number (in REDCap)',nod_num)
                elif all_values[0]!=1: #Could be 2-6 which signifies calcified nodule
                    flag_any=1
                    add_pat(calcified,pat,nod_num,all_slices[0],calcified_vol,all_volumes[0]) #Add to dict with all patients
                    if 'high' in deg: #Add it to dict with only nonemphysema participants
                        add_pat(calcified_high,pat,nod_num,all_slices[0],calcified_vol,all_volumes[0])
                    else: #Add it to dict with only emphysema participants
                        add_pat(calcified_low,pat,nod_num,all_slices[0],calcified_vol,all_volumes[0])
                

                if np.isnan(all_values[2]): #For subsolid/ground glass nodules the 'nodule_type' must be 3 (ground glass) or 2 (partial solid)
                    print('For ', pat, 'we have nan nodule type in nod number (in REDCap)',nod_num)
                elif all_values[2]==3 or all_values[2]==2:
                    flag_any=1
                    add_pat(sub_ground,pat,nod_num,all_slices[0],sub_ground_vol,all_volumes[0])
                    if 'high' in deg:
                        add_pat(sub_ground_high,pat,nod_num,all_slices[0],sub_ground_vol,all_volumes[0])
                    else:
                        add_pat(sub_ground_low,pat,nod_num,all_slices[0],sub_ground_vol,all_volumes[0])
                    
                    
                if np.isnan(all_values[3]): #For atypical we want 'pfn' to be 3 or 4 and 'attachment_7' to be 1
                    print('for ', pat, 'we have nan for pfn_n in nod number (in REDCap)',nod_num)

                elif all_values[3]==3: #If 'pfn' is 3 then 'attachment_7' has to be checked
                #- For 'pfn'=4 (don't know) we agreed to consider them as 'No'
                    
                    flag=1 #That means that we check if it's atypical
                    flag_any=1
                    
                    if ((all_values[8]==3 or all_values[8]==4 or all_values[8]==2 or all_values[8]==1) or all_values[9]==1 #shape triangular, polygonal or oval (or round)
                        and all_values[0]==1): #non-calcified
                    
                        if all_values[4]==0 and np.isnan(all_values[4])==False: #If 'attachment_7' is 0 then atypical
                            add_pat(atypical_triangular,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            if 'high' in deg:
                                add_pat(atypical_triangular_high,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            else:
                                add_pat(atypical_triangular_low,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])

                    
                    else:
                        print('atypical PFN error - either shape or calcification not match even if attachment correct for pat',pat)
                        if all_values[4]==0:# and np.isnan(all_values[4])==False: #If 'attachment_7' is 0 then atypical
                            add_pat(atypical_triangular,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            if 'high' in deg:
                                add_pat(atypical_triangular_high,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            else:
                                add_pat(atypical_triangular_low,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                        else:
                            print("Added as atypical although attributes do not match (attach_7=1) for pat",pat)
                            add_pat(atypical_triangular,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            if 'high' in deg:
                                add_pat(atypical_triangular_high,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            else:
                                add_pat(atypical_triangular_low,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            
                #if 'pfn' is 2 or 3 (typical PFN- possible/atypical pfn) and attachment_7=1 (fissural) then peri/fis
                if all_values[3]==2 or all_values[3]==3: 
                    flag_any=1
                    if ((all_values[8]==3 or all_values[8]==4 or all_values[8]==2 or all_values[8]==1) #shape triangular, polygonal or oval (or round)
                        and all_values[0]==1): #non-calcified
                        
                        if all_values[4]==1: #If attachment_7 is 1 we have typical PFNs
                            add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                            if 'high' in deg:
                                add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            else:
                                add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                                
                        elif all_values[4]==0 and all_values[3]==2: #If 'attachment_7'=0 and 'pfn'=2
                            print('pat:',pat,'nod_id:',nod_num,'. Error! We expected attachment_7 to be 1 given that pfn=2 (typical PFN) but it is 0! Need to check manually')
                            print("For now added in PFN group (periphysural)")

                            add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                            if 'high' in deg:
                                add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            else:
                                add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            
                        elif flag==1: #If we also check for 'atypical' then don't do anything since all scenarios already considered
                            pass
                        else:
                            print('We should never be in here for pat',pat,'with nod id',nod_num,'and values',all_values,'need manual check')
                            
                    else:
                        if all_values[4]==1 and all_values[0]==1: #If attachment_7 is 1 we have typical PFNs and calcified is no
                            add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                            if 'high' in deg:
                                add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            else:
                                add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                                
                        elif all_values[4]==0 and all_values[3]==2 and all_values[0]==1: #If 'attachment_7'=0 and 'pfn'=2 and calcified is no
                            print('pat:',pat,'nod_id:',nod_num,'. Error! We expected attachment_7 to be 1 given that pfn=2 (typical PFN) but it is 0! Need to check manually')
                            print("For now added in PFN group (periphysural)")

                            add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                            if 'high' in deg:
                                add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            else:
                                add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            
                        elif flag==1: #If we also check for 'atypical' then don't do anything since all scenarios already considered
                            print(pat,nod_num,"Classified as typical or atypical PFN but should be checked manually - There are attribute errors!")
                            if all_values[3]==2:
                                print("Classified as typical PFN")
                                add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                                if 'high' in deg:
                                    add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                                else:
                                    add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            else: #only possible value =3 here
                                    print("Classified as atypical PFN")
                                    add_pat(atypical_triangular,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                                    if 'high' in deg:
                                        add_pat(atypical_triangular_high,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                                    else:
                                        add_pat(atypical_triangular_low,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            print("\n")
                        else:
                            print('2.We should never be in here for pat',pat,'with nod id',nod_num,'and values',all_values,'need manual check. Added in PFN group (periphysural)')
                            add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                            if 'high' in deg:
                                add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                            else:
                                add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                
                if np.isnan(all_values[1]): #For pleural nodules 'attachment_2' has to be 1
                    print('for ', pat, 'we have nan in attachment__2 in nod number (in REDCap)',nod_num)
                elif all_values[1]==1:
                    flag_any=1
                    add_pat(pleural,pat,nod_num,all_slices[0],pleural_vol,all_volumes[0])
                    if 'high' in deg:
                        add_pat(pleural_high,pat,nod_num,all_slices[0],pleural_vol,all_volumes[0])
                    else:
                        add_pat(pleural_low,pat,nod_num,all_slices[0],pleural_vol,all_volumes[0])
                    
                #For perinbronchial/bronchiovascular lymph nodes we want 'nodule_type' to be 1 and 'location' central,
                #and 'edge' smooth, and 'shape' triangular or polygonal
                if (all_values[7]==1 and (all_values[8]==3 or all_values[8]==4) and all_values[9]==1 and 
                    all_values[2]==1):
                                
                    #Also 'attachment_4' or 'attachment_5' has to be 1
                    if np.isnan(all_values[5]): 
                        print('for ', pat, 'we have nan in attachment!__4 in nod number (in REDCap)',nod_num)
                    elif all_values[6]==1 or all_values[5]==1:
                        add_pat(peri_bronch,pat,nod_num,all_slices[0],peri_bronch_vol,all_volumes[0])
                        if 'high' in deg:
                            add_pat(peri_bronch_high,pat,nod_num,all_slices[0],peri_bronch_vol,all_volumes[0])
                        else:
                            add_pat(peri_bronch_low,pat,nod_num,all_slices[0],peri_bronch_vol,all_volumes[0])

                    # elif all_values[5]==1:
                    #     print("Peribronchial finding in participant",pat,"with nod_num is",nod_num, '(degree is',deg,'). Excluded in our analysis')

                    else: #for cases in which all attachments are 0 - probably errors
                        if flag_any==0 :
                            print("Attachments empty for pat",pat,"with nod id",nod_num)
                            #if 'pfn' is 'No' or 'I don't know and not in any other nodule category, then add it to 'other nodule'
                            if all_values[3]==1 or all_values[3]==4: 
                                print("We consider as nodule the id",nod_num,"of participant",pat)
                                add_pat(other_all,pat,nod_num,all_slices[0],other_all_vol,all_volumes[0])
                                if 'high' in deg:
                                    add_pat(other_all_high,pat,nod_num,all_slices[0],other_all_vol,all_volumes[0])
                                else:
                                    add_pat(other_all_low,pat,nod_num,all_slices[0],other_all_vol,all_volumes[0])
                            else:#If is pfn and attachments empty then add to typical/atypical
                                if all_values[3]==2:
                                    add_pat(per_fisu,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])   
                                    if 'high' in deg:
                                        add_pat(per_fisu_high,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                                    else:
                                        add_pat(per_fisu_low,pat,nod_num,all_slices[0],per_fisu_vol,all_volumes[0])
                                else: #only possible value =3 here
                                        add_pat(atypical_triangular,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                                        if 'high' in deg:
                                            add_pat(atypical_triangular_high,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                                        else:
                                            add_pat(atypical_triangular_low,pat,nod_num,all_slices[0],atypical_triangular_vol,all_volumes[0])
                            print("\n")
                    flag_any=1
                
                #Check for errors - Cases where we didn't get in any of the conditions above - new categories
                if (all_values[0]==1 and all_values[2]!=3 and all_values[3]!=2 and all_values[3]!=3  and #and all_values[3]!=4
                    all_values[1]==0 and all_values[2]!=2 and
                    all_values[7]!=1 and (all_values[8]!=3 and all_values[8]!=4) and all_values[9]!=1 and all_values[2]!=1):
                    
                        print("None of the above categories for participant",pat,"with nod id",nod_num,"and with values",all_values)
                        print('flag_any is',flag_any)
                        print('flag is',flag)
                        
                flag=0 #Set value to 0 before looping to next nodule     
                
                if flag_any==0:
                
                    #if 'pfn' is 'No' or 'I don't know and not in any other nodule category, then add it to 'other nodule'
                    if all_values[3]==1 or all_values[3]==4:              
                        add_pat(other_all,pat,nod_num,all_slices[0],other_all_vol,all_volumes[0])
                        if 'high' in deg:
                            add_pat(other_all_high,pat,nod_num,all_slices[0],other_all_vol,all_volumes[0])
                        else:
                            add_pat(other_all_low,pat,nod_num,all_slices[0],other_all_vol,all_volumes[0])
                    
                flag_any=0

In [20]:
TP_info_redcap(tp_or_fn='tp')

Those with errors (like '2.We should never be in here for pat ....') should be checked manually!

In [21]:
# high_dict #True nodule ids added in REDCap can be found in these dictionaries

In [66]:
#Define nodule group names
nod_groups=['sub_ground','atypical_triangular','per_fisu','pleural', 'peri_bronch','calcified','other_all']

In [67]:
# #For debugging
# #Print which groups have nodules from particular patient
# for group_nam in nod_groups:
#     try:
#         print('group',group_nam,'is',eval(group_nam+'[845334]'))
#     except:         
#         pass

In [68]:
def errors_check(tp_or_fn='tp'):
    "Function to correct errors in REDCap data entry"
    
    if tp_or_fn=='tp':
        type_extract='true_ids_'
    else:
        type_extract='false_ids_'

    #Check for participants that were not added in any of the categories above, or for particular nodules that were not added

    degrees=['high','low'] #List of degree names

    for deg in degrees: #Loop over each degree
        print("Checking nodules of",deg,'degree....')
        
        for pat_id in eval(type_extract+deg+'_dict'): #Loop over participant with TP nodules of participants of that degree 
            
            nod_ids=eval(type_extract+deg+'_dict'+'[pat_id]') #Get all possible nodules ids for that participant (from 1-10)
            BMI_found=[] #Initialize list to empty to be filled with nodule ids that were not added in any of the lists above
            
            for group in nod_groups: #Loop over each of the nodule groups
                
                BMI_group=eval(group) #Get a variable with that group's dictionary

                if pat_id in BMI_group: #If participant in that dictionary
                    nod_BMI=BMI_group[pat_id] #Get nodule dictionary of that participant
                    for nod_found in nod_BMI: #Loop over nodule ids of that participant
                        BMI_found.append(nod_found) #Add that nodule id in the above list

            missed=list(set(nod_ids) - set(BMI_found)) #Get which nodules not added in the above list (not in any category)


            if len(BMI_found)==len(nod_ids): #All nodule included -those are the correct ones, no need for further check
                pass

            #For those below more manual checks are needed
            else:
                flag=0
                
                if len(BMI_found)==0: #All nodules for that participant belong to a different category
                    print(pat_id,'all nodules',missed,"not added in any category")
                    flag=1

                if len(BMI_found)!=len(np.unique(BMI_found)): #Nodules missed and some that belong to multiple categories
                    print(pat_id,"has",missed,"nodules not added in any category, and nodules with multiple attributes",list(set([i for i in BMI_found if BMI_found.count(i)>1])))
                    flag=1
                    check_elements=list(set([i for i in BMI_found if BMI_found.count(i)>1]))
                    
                if flag==0: #Some nodules of a participant that don't belong to any of the above categories
                    print(pat_id,'nodules',missed,"weren't added in any category")
                flag=0

            if ((len(missed)==0 or len(BMI_found)!=len(np.unique(BMI_found))) and len(BMI_found)!=len(nod_ids) and len(BMI_found)!=0):
                nod_ids=eval(type_extract+deg+'_dict'+'[pat_id]') #Get all possible nodules ids for that participant (from 1-10)
                BMI_found=[] #Initialize list to empty to be filled with nodule ids that were not added in any of the lists above
                print("Deletion starts.......")

                for group in nod_groups[:-1]: #Loop over each of the nodule groups

                    BMI_group=eval(group) #Get a variable with that group's dictionary

                    if pat_id in BMI_group: #Loop over participants in that dictionary
                        nod_BMI=BMI_group[pat_id] #Get nodule dictionary of that participant
                        should_del=[] #list of keys that should be deleted
                        
                        for key,nod_found in nod_BMI.items(): #Loop over nodules of that participant
                            
                                if key in check_elements:
                                    BMI_found.append(key) #Add that nodule in the above list
                                    BMI_found.append(group)

                                    try: #Get volume and add to corresponding dictionary
                                        vol=eval(group+'_vol'+"["+str(pat_id)+"]["+str(key)+"]")
                                        all_slices=eval(group+"["+str(pat_id)+"]["+str(key)+"]")

                                        add_pat(other_all,pat_id,key,all_slices,other_all_vol,vol)
                                        if 'high' in deg:
                                            add_pat(other_all_high,pat_id,key,all_slices,other_all_vol,vol)
                                        else:
                                            add_pat(other_all_low,pat_id,key,all_slices,other_all_vol,vol)
                                            
                                        should_del.append(key)
                                        
                                    except:
                                        print('Error - sth went wrong')
                                        pass
                        
                        for ind_del in should_del: #Delete keys from corresponding dictionaries along with their volumes
                            
                            try:
                                exec("del "+group+"[pat_id][ind_del]")
                                exec("del "+group+"_vol[pat_id][ind_del]")
                                print('Deleted nodule of participant',pat_id,' with id',ind_del,'from group',group)
                                
                                try:
                                    exec("del "+group+"_high[pat_id][ind_del]")
                                    print('Deleted nodule from high group with id',ind_del,'from group',group)
                                except:
                                    exec("del "+group+"_low[pat_id][ind_del]")
                                    print('Deleted nodule of pat',pat_id,' from low group with id',ind_del,'from group',group)
                                
                            except:
                                print('Deletion error')
                                print(traceback.print_exc())
                                pass
            
                print("Here are multiple attributes for",pat_id,":", BMI_found)
                print('\n')

In [22]:
errors_check(tp_or_fn='tp')

Atypical lymph nodes will be considered in the nodule group

In [70]:
#Define nodule group names
nod_groups_only=['sub_ground','pleural', 'calcified','other_all','atypical_triangular'] 
lymph_groups=['per_fisu','peri_bronch'] 

#### Get actual number of TP for nodules and for lymph nodes

In [71]:
for BMI_level in ['_low','_high']: #Loop over low/high BMI degrees
    print("Checking findings of",BMI_level,'degree....')
    
    for nod_group in nod_groups_only: #Loop over nodule groups
        
        #Get total num of values in that group
        exec(nod_group+BMI_level+'_nod_only='+'sum([len(x) for x in '+nod_group+BMI_level+'.values()])') 
        print(nod_group+BMI_level,eval(nod_group+BMI_level+'_nod_only'))
        
        #Get volumes for each group
        exec(nod_group+BMI_level+'_nod_only_30_100=0')
        exec(nod_group+BMI_level+'_nod_only_100_300=0')
        exec(nod_group+BMI_level+'_nod_only_300=0')
        
        for pat in eval(nod_group+BMI_level+'.keys()'): #Loop over participants of that group
            exec(nod_group+BMI_level+'_nod_only_30_100+='+'len([1 for val in '+nod_group+'_vol[pat].values() if val>=30 and val<100])')
            exec(nod_group+BMI_level+'_nod_only_100_300+='+'len([1 for val in '+nod_group+'_vol[pat].values() if val>=100 and val<=300])')
            exec(nod_group+BMI_level+'_nod_only_300+='+'len([1 for val in '+nod_group+'_vol[pat].values() if val>300])')
            
        print(nod_group+BMI_level+'30_100',eval(nod_group+BMI_level+'_nod_only_30_100'))
        print(nod_group+BMI_level+'100-300',eval(nod_group+BMI_level+'_nod_only_100_300'))
        print(nod_group+BMI_level+'300+',eval(nod_group+BMI_level+'_nod_only_300'))
        print("\n")
        
        #Save them to be used in the other file to create tables with these numbers           
        with open(nod_group+BMI_level+'_nod_only'+'.pickle','wb') as f:
            pickle.dump(eval(nod_group+BMI_level+'_nod_only'),f)          
            
        with open(nod_group+BMI_level+'_nod_only_30_100'+'.pickle','wb') as f:
            pickle.dump(eval(nod_group+BMI_level+'_nod_only_30_100'),f)
        with open(nod_group+BMI_level+'_nod_only_100_300'+'.pickle','wb') as f:
            pickle.dump(eval(nod_group+BMI_level+'_nod_only_100_300'),f)
        with open(nod_group+BMI_level+'_nod_only_300'+'.pickle','wb') as f:
            pickle.dump(eval(nod_group+BMI_level+'_nod_only_300'),f)
     
    #Same as above for lymph nodes        
    for lymph_group in lymph_groups:
        
        exec(lymph_group+BMI_level+'_lymph='+'sum([len(x) for x in '+lymph_group+BMI_level+'.values()])')
        print(lymph_group+BMI_level,eval(lymph_group+BMI_level+'_lymph'))
        
        #Get volumes for each group
        exec(lymph_group+BMI_level+'_lymph_30_100=0')
        exec(lymph_group+BMI_level+'_lymph_100_300=0')
        exec(lymph_group+BMI_level+'_lymph_300=0')
        
        for pat in eval(lymph_group+BMI_level+'.keys()'):
            exec(lymph_group+BMI_level+'_lymph_30_100+='+'len([1 for val in '+lymph_group+'_vol[pat].values() if val>=30 and val<100])')
            exec(lymph_group+BMI_level+'_lymph_100_300+='+'len([1 for val in '+lymph_group+'_vol[pat].values() if val>=100 and val<=300])')
            exec(lymph_group+BMI_level+'_lymph_300+='+'len([1 for val in '+lymph_group+'_vol[pat].values() if val>300])')
     
        print(lymph_group+BMI_level+'30_100',eval(lymph_group+BMI_level+'_lymph_30_100'))
        print(lymph_group+BMI_level+'100-300',eval(lymph_group+BMI_level+'_lymph_100_300'))
        print(lymph_group+BMI_level+'300+',eval(lymph_group+BMI_level+'_lymph_300'))
        print("\n")

        with open(lymph_group+BMI_level+'_lymph'+'.pickle','wb') as f:
            pickle.dump(eval(lymph_group+BMI_level+'_lymph'),f)
    
        with open(lymph_group+BMI_level+'_lymph_30_100'+'.pickle','wb') as f:
            pickle.dump(eval(lymph_group+BMI_level+'_lymph_30_100'),f)
        with open(lymph_group+BMI_level+'_lymph_100_300'+'.pickle','wb') as f:
            pickle.dump(eval(lymph_group+BMI_level+'_lymph_100_300'),f)
        with open(lymph_group+BMI_level+'_lymph_300'+'.pickle','wb') as f:
            pickle.dump(eval(lymph_group+BMI_level+'_lymph_300'),f)

    print("\n")

Checking findings of _low degree....
sub_ground_low 1
sub_ground_low30_100 1
sub_ground_low100-300 0
sub_ground_low300+ 0


pleural_low 12
pleural_low30_100 8
pleural_low100-300 2
pleural_low300+ 2


calcified_low 9
calcified_low30_100 9
calcified_low100-300 0
calcified_low300+ 0


other_all_low 37
other_all_low30_100 26
other_all_low100-300 9
other_all_low300+ 2


atypical_triangular_low 5
atypical_triangular_low30_100 5
atypical_triangular_low100-300 0
atypical_triangular_low300+ 0


per_fisu_low 22
per_fisu_low30_100 17
per_fisu_low100-300 5
per_fisu_low300+ 0


peri_bronch_low 1
peri_bronch_low30_100 0
peri_bronch_low100-300 0
peri_bronch_low300+ 1




Checking findings of _high degree....
sub_ground_high 1
sub_ground_high30_100 0
sub_ground_high100-300 0
sub_ground_high300+ 1


pleural_high 8
pleural_high30_100 6
pleural_high100-300 2
pleural_high300+ 0


calcified_high 6
calcified_high30_100 6
calcified_high100-300 0
calcified_high300+ 0


other_all_high 36
other_all_high30_100 2

In [72]:
#Print total num of values of TPs in each group
print("High BMI total findings:",sum([len(x) for x in high_dict.values()]))
print("Low BMI total findings:",sum([len(x) for x in low_dict.values()]))

print("High BMI number of participants:",(len(high_dict.keys())))
print("Low BMI number of participants:",(len(low_dict.keys())))

High BMI total findings: 67
Low BMI total findings: 87
High BMI number of participants: 46
Low BMI number of participants: 59


#### Get ids of FN to automatically perform analysis - Similar as above for TPs

In [73]:
#Select rows where we have at least one FN in any of the 0-100, 100-300 or 300+mm3 volume subgroup
high_fn=high[(high['100-300fn'].notnull() | high['0-100fn'].notnull() | high['300+ fn'].notnull()) & high['participant_id'].notnull()] 
low_fn=low[(low['100-300fn'].notnull() | low['0-100fn'].notnull() | low['300+ fn'].notnull()) & low['participant_id'].notnull()] 

In [23]:
# high_fn

In [75]:
#Get lists of participants with at least one FN
high_pats_fn=[int(str(x)[:6]) for x in high_fn['participant_id'].values]
low_pats_fn=[int(str(x)[:6]) for x in low_fn['participant_id'].values]

In [76]:
#Get only rows from REDCap dataframe with participants that have at least on FN
high_df_fn=TP_export[TP_export['participant_id'].isin(high_pats_fn)]
low_df_fn=TP_export[TP_export['participant_id'].isin(low_pats_fn)]

In [24]:
# high_df_fn

In [25]:
# low_df_fn

In [79]:
#Initialize empty dicts in the form {'pat_id1':[],'pat_id2':[],...} mod_df_fn['participant_id']

high_dict_fn=dict.fromkeys([int(str(numeric_string)[:6]) for numeric_string in high_df_fn['participant_id'].values], [])
low_dict_fn=dict.fromkeys([int(str(numeric_string)[:6]) for numeric_string in low_df_fn['participant_id'].values], [])

#Initialize same dict for storing volumes of each FN nodule
high_dict_fn_vol=high_dict_fn.copy()
low_dict_fn_vol=low_dict_fn.copy()

In [80]:
for deg in BMI_deg: #Loop over low/high BMI degrees (in the form 'low_tp')

    for ind,pat in enumerate(eval(deg[:-3]+"_df_fn['participant_id']")): #Loop over all indices and participants of a given df
        if pat in eval(deg[:-3]+"_dict.keys()"): #If the participant is also in dict with some nodule ids being TP
            
            for nod_id,val in enumerate(eval(deg[:-3]+"_df_fn[cols_with_nod_ids].iloc[ind].values")): #Loop over all ids

                if nod_id!=0: #Avoid first column in which we might have both 'n1' and 'n10'
                    vol_cols=[col for col in TP_export.columns if 'volume_all_n'+str(nod_id+1) in col]
                else:
                    vol_cols=[col for col in TP_export.columns if 'volume_all_n'+str(nod_id+1) in col and '_n10' not in col]        

                #If a specific nodule id is not in the dict with TP add it to dict with FN. Same for its volume
                #First condition will result in error in pat not in that dict - That's why we need the 'else' condition below
                if val not in eval(deg[:-3]+"_dict[pat]") and np.isnan(val)==False: 
                    exec(deg[:-3]+"_dict_fn[pat]="+deg[:-3]+"_dict_fn[pat]+[val]")
                    exec(deg[:-3]+"_dict_fn_vol[pat]="+deg[:-3]+"_dict_fn_vol[pat]+["+deg[:-3]+"_df_fn[vol_cols].iloc[ind].values[0]]")
                        
        else: #If this is not in dictionary with at least one TP then all are FNs

            for nod_id,val in enumerate(eval(deg[:-3]+"_df_fn[cols_with_nod_ids].iloc[ind].values")):

                if nod_id!=0: #Avoid first column in which we might have both 'n1' and 'n10'
                    vol_cols=[col for col in TP_export.columns if 'volume_all_n'+str(nod_id+1) in col]
                else:
                    vol_cols=[col for col in TP_export.columns if 'volume_all_n'+str(nod_id+1) in col and '_n10' not in col]        

                if np.isnan(val)==False:
                    exec(deg[:-3]+"_dict_fn[pat]="+deg[:-3]+"_dict_fn[pat]+[val]")
                    exec(deg[:-3]+"_dict_fn_vol[pat]="+deg[:-3]+"_dict_fn_vol[pat]+["+deg[:-3]+"_df_fn[vol_cols].iloc[ind].values[0]]")

In [81]:
false_ids_high_dict={key:[] for key,val in high_dict_fn.items()} 
false_ids_low_dict={key:[] for key,val in low_dict_fn.items()} 

#Apply function to get nodule id in REDCap from nodule value stored in it
false_ids_high_dict=convert_to_true_ids(high_dict_fn,high_df_fn,cols_with_nod_ids,false_ids_high_dict)
false_ids_low_dict=convert_to_true_ids(low_dict_fn,low_df_fn,cols_with_nod_ids,false_ids_low_dict)

In [26]:
# low_dict_fn

In [27]:
# false_ids_low_dict

In [28]:
# high_dict_fn

In [29]:
# false_ids_high_dict

In [86]:
#Print total num of values of FNs in each group
print('Number of participants with High BMI FN is',len(high_dict_fn))
print('Number of participants with Low BMI FN is',len(low_dict_fn))

print("Number of findings in High BMI FN is",len([y for x in high_dict_fn.values() for y in x])) #Same as sum([len(x) for x in high_dict_fn.values()])
print("Number of findings in Low BMI FN is",len([y for x in low_dict_fn.values() for y in x]))

Number of participants with High BMI FN is 28
Number of participants with Low BMI FN is 38
Number of findings in High BMI FN is 42
Number of findings in Low BMI FN is 55


In [87]:
print("In High BMI there are in total", len(np.unique(list(high_dict_fn.keys())+list(high_dict.keys()))), 'participants with nodules (either TP or FN)')
print("In Low BMI there are in total", len(np.unique(list(low_dict_fn.keys())+list(low_dict.keys()))), 'participants with nodules (either TP or FN)')

In High BMI there are in total 60 participants with nodules (either TP or FN)
In Low BMI there are in total 78 participants with nodules (either TP or FN)


#### Get IDs and slices of FN per type - Extract type of findings for those from REDCap

In [30]:
# false_ids_low_dict

In [31]:
# atypical_triangular_low

In [32]:
TP_info_redcap(tp_or_fn='fn')

In [33]:
# atypical_triangular_low

In [34]:
# other_all

In [35]:
errors_check(tp_or_fn='fn')

Findings 'weren't added in any category' could be due to being peribronchial

In [95]:
#Count total num of FN in those lists
for bmi in ['_low','_high']: #Loop over low/high BMI degrees

    print("Checking findings of",bmi,'degree....')
    
    exec('total'+bmi+'=0') #Count

    for nod_group in nod_groups_only: #Loop over nodule groups
        total_vals=np.sum([len(x) for x in eval(nod_group+bmi+'.values()')])
        exec('total'+bmi+'+=total_vals') #Add to total
        print(nod_group+bmi,total_vals)
    for lymph_group in lymph_groups:
        total_vals=np.sum([len(x) for x in eval(lymph_group+bmi+'.values()')])
        print(lymph_group+bmi,total_vals) #Add to total
        exec('total'+bmi+'+=total_vals')

    print("Total",bmi,eval('total'+bmi))
    print("\n")

Checking findings of _low degree....
sub_ground_low 0.0
pleural_low 3
calcified_low 1
other_all_low 29
atypical_triangular_low 1
per_fisu_low 13
peri_bronch_low 8
Total _low 55.0


Checking findings of _high degree....
sub_ground_high 0.0
pleural_high 2
calcified_high 1
other_all_high 19
atypical_triangular_high 2
per_fisu_high 11
peri_bronch_high 7
Total _high 42.0




Above are those detected only by reader - could be nodules, lymphs or non-nodules based on consensus - should be manually checked below

#### Compare initial reading vs consensus for FN

In [96]:
#Get all FNs in a list
for bmi in ['low','high']: #Loop over low/high BMI degrees

    print("Checking findings of",bmi,'degree....')
    
    exec(bmi+"_FN_all={}") 
    exec(bmi+"_FN_all_vols={}") 

    for pat in eval("false_ids_"+bmi+"_dict"+'.keys()'):

        exec(bmi+"_FN_all"+"["+str(pat)+"]={}") #Initialize empty dict for each participant
        exec(bmi+"_FN_all_vols"+"["+str(pat)+"]={}") #Initialize empty dict for each participant

        for fn_id in eval("false_ids_"+bmi+"_dict"+'['+str(pat)+']'):

            slice_val=TP_export[TP_export['participant_id']==pat]['slice_number_n'+str(fn_id)].values[0]
            exec(bmi+"_FN_all["+str(pat)+"][fn_id]=slice_val") #Add slice number of each FN nodule

            vol_val=TP_export[TP_export['participant_id']==pat]['volume_solid_n'+str(fn_id)].values[0]
            if np.isnan(vol_val):
                vol_val=TP_export[TP_export['participant_id']==pat]['volume_subsolid_n'+str(fn_id)].values[0]
                # if np.isnan(vol_val):
                #     print("NaN volume for",pat,fn_id)

            exec(bmi+"_FN_all_vols["+str(pat)+"][fn_id]=vol_val") #Add slice number of each FN nodule

Checking findings of low degree....
Checking findings of high degree....


In [36]:
# low_FN_all

In [37]:
# low_FN_all_vols

Extract information from consensus review - For now should compare manually the two dictionaries below

In [99]:
# Low/High BMI

for deg in ['low','high']:
    exec(deg+"_consensus=copy.deepcopy("+deg+'_FN_all)') #Create a copy of the above dictionary

    for key,val in eval(deg+"_consensus.items()"): 
        for id,slice_num in val.items():
            exec(deg+"_consensus[key][id]={slice_num:{}}") #Add slice number as key for each id


    for folder in os.listdir(eval("path_"+deg)): #Loop over folders in which we have txt files with consensus review
        if os.path.isdir(eval('path_'+deg)+'/'+folder):

            folder_name=folder[:6]

            for file in os.listdir(eval('path_'+deg)+'/'+folder):
                if 'fn' in file and 'txt' in file:
                    with open (eval('path_'+deg)+'/'+folder+'/'+file,'r') as f:
                        lines=f.readlines() #Read lines of txt file


                    pat_fn=eval(deg+'_FN_all[int(folder_name)]') #Get dictionary with slice numbers of FNs for that participant

                    for id,slice_num in pat_fn.items(): #Loop over slice numbers of FNs for that participant
                        if str(int(slice_num)) in file: #If slice number of FN is in file name

                            for pat,vals in eval(deg+'_FN_all.items()'): #Loop over all participants with FNs
                                if pat==int(folder_name): #If participant is the same as the one in file name
                                    for id_num in vals.keys(): #Loop over all ids of that participant
                                        
                                        if int(eval(deg+'_FN_all[int(folder_name)][id_num]==int(slice_num)')): 
                                            exec(deg+'_consensus[int(folder_name)][id_num][slice_num]=lines')
                                            #Add lines of txt file as value for that slice number

In [38]:
# high_FN_all

In [39]:
# low_consensus

In [40]:
# low_FN_all

In [41]:
# low_FN_all_vols

In [42]:
# high_consensus

In [43]:
# high_FN_all

In [44]:
# high_FN_all_vols

In [107]:
#Function to print in color
def prRed(skk): print("\033[91m {}\033[00m".format(skk))
#https://www.geeksforgeeks.org/print-colors-python-terminal/

#Possible colors are:
# red = '\033[91m'
# green = '\033[92m'
# yellow = '\033[93m'
# blue = '\033[94m'
# pink = '\033[95m'
# teal = '\033[96m'
# grey = '\033[97m'

In [45]:
#low/high BMI final comparison of consensus vs REDCap
for bmi in ['low','high']: #Loop over emphysema degrees

    show_deg="Degree is "+bmi+'.....'
    prRed(show_deg) #Print in red
    all_count=0

    for pat in eval(bmi+'_consensus'): #Loop over participants with FNs in consensus review
        for id in eval(bmi+'_consensus[pat]'): #Loop over nodule ids of that participant
            
            for slice_num in eval(bmi+'_consensus['+str(pat)+']['+str(id)+']'): #Loop over slice numbers of that nodule

                #Count total num of FN in those lists
                for type_bmi in ['_low','_high']: #Loop over emphysema/non-emphysema degrees
                    for nod_group in nod_groups_only: #this is based on redcap attibutes and not on consensus review
                        try: #If slice number is in that dictionary
                            if slice_num in eval(nod_group+type_bmi+'[pat].values()'): 
                                for check_id,check_val in eval(nod_group+type_bmi+'[pat].items()'): #Loop over all ids of that participant
                                    if slice_num==check_val: #If slice number is the same as the one in dictionary
                                        
                                        vol_val=eval(bmi+'_FN_all_vols[pat][id]')

                                        print("Participant",pat,'with bmi:',bmi,"nodule id:",id,"slice number:",slice_num,"volume:",vol_val)
                                        print("REDCap:",nod_group)
                                        print("Consensus:",(eval(bmi+'_consensus['+str(pat)+']['+str(id)+']['+str(slice_num)+']')))
                                        all_count+=1
                                        print("\n")
                        except:
                            pass

                    for lymph_group in lymph_groups: #this is based on redcap attibutes and not on consensus review
                        try:
                            if slice_num in eval(lymph_group+type_bmi+'[pat].values()'):
                                for check_id,check_val in eval(lymph_group+type_bmi+'[pat].items()'):
                                    if slice_num==check_val:

                                        vol_val=eval(bmi+'_FN_all_vols[pat][id]')

                                        print("Participant",pat,'with bmi:',bmi,"nodule id:",id,"slice number:",slice_num,"volume:",vol_val)
                                        print("REDCap:",lymph_group)
                                        print("Consensus:",(eval(bmi+'_consensus['+str(pat)+']['+str(id)+']['+str(slice_num)+']')))
                                        all_count+=1
                                        print("\n")
                        except:
                            pass

    print("Total findings:",all_count)

#### Get actual slice with those ids - To be used to match nodule slices from review sessions

#### Cells below only load dictionaries created from first part of file 'reviewing_info_extract.ipynb'. This is why it is suggested to run that file instead of this, since there, this file is called and it works properly.

In [109]:
slice_cols=[col for col in TP_export.columns if 'slice_number_n' in col] #Get columns with slice numbers
vol_cols=[col for col in TP_export.columns if 'volume_all_n' in col] #Get columns with volumes

In [110]:
#Load dictionaries that contain all nodules and lymph nodes
with open('dict_FN_wrong_low.pickle', 'rb') as f:
    dict_FN_wrong_low = pickle.load(f)

with open('dict_FN_correct_low.pickle', 'rb') as f:
    dict_FN_correct_low = pickle.load(f)

with open('dict_FN_wrong_high.pickle', 'rb') as f:
    dict_FN_wrong_high = pickle.load(f)
    
with open('dict_FN_correct_high.pickle', 'rb') as f:
    dict_FN_correct_high = pickle.load(f)
    
#Similarly for dictionaries having only nodules or only lymph nodes
with open('lymph_FN_correct_low.pickle','rb') as f:
    lymph_FN_correct_low=pickle.load(f)    

with open('lymph_FN_correct_high.pickle','rb') as f:
    lymph_FN_correct_high=pickle.load(f) 
    
with open('nod_FN_correct_low.pickle','rb') as f:
    nod_FN_correct_low=pickle.load(f)    

with open('nod_FN_correct_high.pickle','rb') as f:
    nod_FN_correct_high=pickle.load(f) 

In [47]:
# dict_FN_wrong_low

In [48]:
# dict_FN_correct_low

In [49]:
#Get copies of the dictionaries above to match slices with their indices

#Get list of strings with the names of the above loaded dictionaries
all_FNs=['dict_FN_wrong_low','dict_FN_correct_low','dict_FN_wrong_high','dict_FN_correct_high',
         'lymph_FN_correct_low','lymph_FN_correct_high','nod_FN_correct_low','nod_FN_correct_high']

count_del=0 #count the number of elements to be deleted

for FN_dict in all_FNs: #Loop over above dictionaries with FNs

    #Get copies of dictionaries with ids, and volumes for those
    exec(FN_dict+'_ids=copy.deepcopy('+FN_dict+')')
    exec(FN_dict+'_ids={item: [] for item in '+FN_dict+'_ids}')
    exec(FN_dict+'_vols=copy.deepcopy('+FN_dict+'_ids)') #Get empty lists to fill in volumes

    exec(FN_dict+'_errors=[]') #Empty list to keep track of errors
    
    for key,slices in eval(FN_dict+'.items()'): #Loop over elements of above dictionaries

        df_values=list(TP_export[slice_cols][TP_export['participant_id']==int(key)].values[0]) #Get values from id 1-10
        df_volumes=list(TP_export[vol_cols][TP_export['participant_id']==int(key)].values[0]) #Get their volumes

        for slice_num in slices: #Loop over the slices from the reviewing session
            if df_values.count(slice_num)==1: #If this slice exists exactly one time
                 for ind,val in enumerate(df_values): #Loop over the slice ids
                        if val==slice_num: #When we fing this slice keep its index
                            exec(FN_dict+'_ids[key].append(ind+1)') #Add it to dictionary
                            exec(FN_dict+'_vols[key].append(df_volumes[ind])') #Same for volume

            else: #These should be checked manually
                print("Dictionary is",FN_dict)
                print("Slices are",slices)
                print("Num of occurences is",df_values.count(slice_num))
                print("In participant",key,"the slice",slice_num,"exists more than once - Should be checked manually")
                print("Values of the slices for that participant are:",df_values)
                
                exec(FN_dict+'_errors.append(key)')

    #Delete participants for which we have the same value more than once - probably wrong information added there
    for par in eval(FN_dict+'_errors'):
        
        if 'dict' in FN_dict: 
            try: #Since it might not exist
                count_del=count_del+len(eval(FN_dict+'_ids[par]'))
            except:
                print('empty participant:',par)
            

        try: #In case it does not exist
            print("Going to delete dictionary", FN_dict,"with ids",eval(FN_dict+'_ids[par]'),'of participant',par)

            exec(FN_dict+'_ids.pop(par)')
            exec(FN_dict+'_vols.pop(par)')
        except:
            print("Deletion Failed for",par)
        print('\n')

print("Total number of elements deleted from main dictionary is",count_del)

The above should be added manually, if needed

We changed atypical PFNs to nodule group - IDs should be checked to confirm that - Should be fine, added to corresponding group based on consensus panel review.

In [114]:
#Add manually slices, ids, and volumes to corresponding dictionary for cases found above 
dict_FN_correct_low_vols['123097']=[54.0] 
lymph_FN_correct_low_vols['123097']=[54.0]

dict_FN_correct_low_ids['123097']=[6]
lymph_FN_correct_low_ids['123097']=[6]

Below are not used anymore - Manually modified excels

In [116]:
#Save all dictionaries to be used in the other file to match slices with ids
with open('dict_FN_wrong_low_ids.pickle','wb') as f:
    pickle.dump(dict_FN_wrong_low_ids,f)

with open('dict_FN_correct_low_ids.pickle','wb') as f:
    pickle.dump(dict_FN_correct_low_ids,f)    

with open('dict_FN_wrong_high_ids.pickle','wb') as f:
    pickle.dump(dict_FN_wrong_high_ids,f)

with open('dict_FN_correct_high_ids.pickle','wb') as f:
    pickle.dump(dict_FN_correct_high_ids,f) 
    
#Same for volumes of the above files
with open('dict_FN_wrong_low_vols.pickle','wb') as f:
    pickle.dump(dict_FN_wrong_low_vols,f)

with open('dict_FN_correct_low_vols.pickle','wb') as f:
    pickle.dump(dict_FN_correct_low_vols,f)    

with open('dict_FN_wrong_high_vols.pickle','wb') as f:
    pickle.dump(dict_FN_wrong_high_vols,f)

with open('dict_FN_correct_high_vols.pickle','wb') as f:
    pickle.dump(dict_FN_correct_high_vols,f) 
    
#Same as above for lymph node ids and volumes
with open('lymph_FN_correct_low_ids.pickle','wb') as f:
    pickle.dump(lymph_FN_correct_low_ids,f)  

with open('lymph_FN_correct_high_ids.pickle','wb') as f:
    pickle.dump(lymph_FN_correct_high_ids,f)
    
with open('lymph_FN_correct_low_vols.pickle','wb') as f:
    pickle.dump(lymph_FN_correct_low_vols,f) 

with open('lymph_FN_correct_high_vols.pickle','wb') as f:
    pickle.dump(lymph_FN_correct_high_vols,f)
    
#Same for nodule dictionaries
with open('nod_FN_correct_low_ids.pickle','wb') as f:
    pickle.dump(nod_FN_correct_low_ids,f)  

with open('nod_FN_correct_high_ids.pickle','wb') as f:
    pickle.dump(nod_FN_correct_high_ids,f)

with open('nod_FN_correct_low_vols.pickle','wb') as f:
    pickle.dump(nod_FN_correct_low_vols,f) 

with open('nod_FN_correct_high_vols.pickle','wb') as f:
    pickle.dump(nod_FN_correct_high_vols,f)

#### Analysis per number of nodules/sizes and Plots

In [117]:
conditions = (high[['0-100tp', '100-300tp', '300+ tp', '0-100fn', '100-300fn', '300+ fn']] == 0) | high[['0-100tp', '100-300tp', '300+ tp', '0-100fn', '100-300fn', '300+ fn']].isna()
high_nonnods= conditions.all(axis=1).astype(int)
print("There are",(high_nonnods == 0).sum()-1,'participants with at least one nodule in high BMI group')
print("There are",(high_nonnods == 1).sum(),"participants with no nodules in high BMI group") #-1 to remove last row with total nodules

conditions = (low[['0-100tp', '100-300tp', '300+ tp', '0-100fn', '100-300fn', '300+ fn']] == 0) | low[['0-100tp', '100-300tp', '300+ tp', '0-100fn', '100-300fn', '300+ fn']].isna()
low_nonnods= conditions.all(axis=1).astype(int)
print("There are",(low_nonnods == 0).sum()-1,'participants with at least one nodule in low BMI group')
print("There are",(low_nonnods == 1).sum(),"participants with no nodules in low BMI group") #-1 to remove last row with total nodules

There are 60 participants with at least one nodule in high BMI group
There are 116 participants with no nodules in high BMI group
There are 78 participants with at least one nodule in low BMI group
There are 98 participants with no nodules in low BMI group


In [118]:
print("There are",int(high.iloc[-1,:]['0-100tp']+high.iloc[-1,:]['100-300tp']+high.iloc[-1,:]['300+ tp']+high.iloc[-1,:]['0-100fn']+high.iloc[-1,:]['100-300fn']), 'nodules in high BMI group')
#+high.iloc[-1,:]['300+ fn'] is nan
print("There are", int(np.sum(high[['0-100tp']].iloc[:-1])[0])+int(np.sum(high[['0-100fn']].iloc[:-1])[0]),'nodules of size 30-100mm3 in high BMI group')
print("There are", int(np.sum(high[['100-300tp']].iloc[:-1])[0])+int(np.sum(high[['100-300fn']].iloc[:-1])[0]),'nodules of size 100-300mm3 in high BMI group')
print("There are", int(np.sum(high[['300+ tp']].iloc[:-1])[0])+int(np.sum(high[['300+ fn']].iloc[:-1])[0]),'nodules of size 300+mm3 in high BMI group')

print("There are",int(low.iloc[-1,:]['0-100tp']+low.iloc[-1,:]['100-300tp']+low.iloc[-1,:]['300+ tp']+low.iloc[-1,:]['0-100fn']+low.iloc[-1,:]['100-300fn']+low.iloc[-1,:]['300+ fn']), 'nodules in low BMI group')
print("There are", int(np.sum(low[['0-100tp']].iloc[:-1])[0])+int(np.sum(low[['0-100fn']].iloc[:-1])[0]),'nodules of size 30-100mm3 in low BMI group')
print("There are", int(np.sum(low[['100-300tp']].iloc[:-1])[0])+int(np.sum(low[['100-300fn']].iloc[:-1])[0]),'nodules of size 100-300mm3 in low BMI group')
print("There are", int(np.sum(low[['300+ tp']].iloc[:-1])[0])+int(np.sum(low[['300+ fn']].iloc[:-1])[0]),'nodules of size 300+mm3 in low BMI group')

There are 109 nodules in high BMI group
There are 87 nodules of size 30-100mm3 in high BMI group
There are 18 nodules of size 100-300mm3 in high BMI group
There are 4 nodules of size 300+mm3 in high BMI group
There are 142 nodules in low BMI group
There are 114 nodules of size 30-100mm3 in low BMI group
There are 22 nodules of size 100-300mm3 in low BMI group
There are 6 nodules of size 300+mm3 in low BMI group


# Below not correct here since they also consider nodules changed above to have '0' volume.
May also be related to 588074 being 588047!

In [119]:
#For a specific degree of emphysema, get all nodule sizes, a dictionary with the frequency of the number of nodules
#that appear in the patients of that dataframe, a list with the name of the patient that each nodule corresponds to
#(found on sizes), and the number of nodules found on that patient
#plotting parameter should be set to False when creating plots so that we take into account nodules >300mm3

def get_nodule_names_and_sizes(dataframe,plotting=False): #Input is a dataframe of a specific degree of emphysema
    
    #Initialize empty lists to be filled with nodule sizes, patient names, and num of nodules of each patient
    sizes=[]
    patient_names=[]
    patient_nodules=[]
    
    all_nodules_num=[] #Keep track of the number of nodules 
    
    for name in dataframe.columns: #There are many columns with nodules, one column for each unique nodule
        if 'volume_all' in name: #these are columns with nodules
            count = dataframe[name].isna().sum() #count NaN values
            print("Column {} has {} non-nan values".format(name,len(dataframe)-count))
            
            selected_rows = dataframe[~dataframe[name].isna()] #select rows with non nan values
            sizes.extend(selected_rows[name].values.tolist()) #add the values of these nodules to a list
            patient_names.extend(selected_rows['participant_id'].values.tolist()) #add patient_ids to a list
            patient_nodules.extend(selected_rows['measured_nodules'].values.tolist()) #add number of nodules to a list
        
            if plotting==True:
                #if we don't rename it then wrong indices will be added in patient_names and nodules
                sizes_new=[val for ind,val in enumerate(sizes) if ind in np.where(np.array(sizes)<=300)[0]]
                patient_names=[patient_names[ind] for ind,val in enumerate(sizes) if ind in np.where(np.array(sizes)<=300)[0]]
                patient_nodules=[patient_names.count(x) for x in patient_names]
                sizes=sizes_new

    all_nodules_num.extend(patient_nodules) #Extend a list with the number of nodules counted in each participant
            
    num_nodules=dataframe['measured_nodules'].values #Get all the values of the number of nodules for each patient, even 0
    
    if plotting==False:
        num_nodules_no_zero=num_nodules[num_nodules!=0] #Get num of nodules if nodules detected
    else:
        num_nodules_no_zero=all_nodules_num #Replace with number of nodules created above - only keep those <300mm3

    non_zero_count={} #create a dictionary with the number of nodules and how often that number is detected
    for elem in num_nodules_no_zero:
        if elem in non_zero_count:
            non_zero_count[elem]=non_zero_count[elem]+1
        else:
            non_zero_count[elem]=1
        
    if plotting==True:
        for key,val in non_zero_count.items(): #Added to make sure that the right number was counted
            non_zero_count[key]=np.ceil(val/key)    
    
        
    non_zero_count[0.0]=num_nodules.tolist().count(0) #add at the end the case when no nodules were detected

    print("\n")
    print("Dictionary with number of measured nodules and how many times that number occured")
    
    non_zero_count_sorted = dict(sorted(non_zero_count.items())) #sorted dictionary
    print(non_zero_count_sorted)
    print('\n')
    
    print("All participants",patient_names)
    
    return sizes, non_zero_count_sorted, patient_names, patient_nodules

In [120]:
new_severity_names=['high_redcap','low_redcap']

In [121]:
#BMI types ordered by severity
severity=['high_redcap','low_redcap']
severity_nums=[1,0] #create integers from the most severe to the least one 
severity_dict={} #create a dict with severity name and corresponding integer
for index,val in enumerate(severity):
    severity_dict[val]=severity_nums[index]
severity_dict

{'high_redcap': 1, 'low_redcap': 0}

In [50]:
#Barplots for different degrees of emphysema
fig, axs = plt.subplots(ncols=len(severity_dict),figsize=(17,4)) #set to 23,6 if trace and mild are included
fig2, axs2 = plt.subplots(ncols=len(severity_dict),figsize=(17,4)) 
# fig3, axs3 = plt.subplots(ncols=len(severity_dict),figsize=(17,4)) 
# fig_4, axs_4 = plt.subplots(ncols=len(severity_dict),figsize=(17,4))

for ind,name in enumerate(severity_dict.keys()): #loop over names of emphysema degrees
    print(name)
    
    #Execute above function to get nodule names and sizes for each emphysema degree
    exec("sizes_"+new_severity_names[ind]+',nodule_num_'+new_severity_names[ind]+',patient_names_'+new_severity_names[ind]+
         ',patient_nodules_'+new_severity_names[ind]+'=get_nodule_names_and_sizes('+name+',False)') 
       
    #Barplot of the distribution of the num of nodules for each degree of emphysema using key-value pairs from dictionaries   
    freq=np.asarray(list(eval('nodule_num_'+new_severity_names[ind]+'.values()')))
    vals=np.asarray(list(eval('nodule_num_'+new_severity_names[ind]+'.keys()')))
    fig=sns.barplot(vals,freq,ax=axs[ind])
    fig.set(xlabel=new_severity_names[ind]+'_num_of_nodules', ylabel='num_of_individuals')
    
    #Histogram of sizes when outliers were removed   
    sizes_no_outliers = [x for x in eval('sizes_'+new_severity_names[ind]) if x <=300 and x>0]  
    s = pd.DataFrame({'val': sizes_no_outliers, 'color':['<=100$mm^3$' if x<=100 else '>100$mm^3$ & <=300$mm^3$' for x in sizes_no_outliers]})
    fig2=sns.histplot(data=s,ax=axs2[ind],x='val',hue='color',hue_order=["<=100$mm^3$", ">100$mm^3$ & <=300$mm^3$"])
#     for ax in axs2: #delete legend
#         ax.legend([],[], frameon=False)
    fig2.set(xlabel=new_severity_names[ind]+'_sizes ($mm^3$)', ylabel='num_of_nodules')
    
#     #The above was originally as:
#     sizes_no_outliers = [x for x in eval('sizes_'+new_severity_names[ind]) if x <=300]  
#     fig2=sns.histplot(sizes_no_outliers,ax=axs2[ind])
#     fig2.set(xlabel=new_severity_names[ind]+'_sizes ($mm^3$)', ylabel='num_of_nodules')
    
#     #Histogram of all sizes, even outliers
#     vals=np.asarray(eval('sizes_'+new_severity_names[ind]))
#     fig3=sns.histplot(vals,ax=axs3[ind])
#     fig3.set(xlabel=new_severity_names[ind]+'_sizes_all ($mm^3$)', ylabel='num_of_nodules')
    
    #A Distplot or distribution plot, depicts the variation in the data distribution. 
    #Seaborn Distplot represents the overall distribution of continuous data variables.
    #Same graph as above with a distribution line fit
#     vals=np.asarray(eval('sizes_'+new_severity_names[ind]))
#     fig_4=sns.distplot(vals,ax=axs_4[ind])
#     fig_4.set(xlabel=new_severity_names[ind]+'_sizes_all ($mm^3$3)', ylabel='num_of_nodules')


#Save each of the above graphs seperately
# bar_num_nods = fig.get_figure()
# bar_num_nods.savefig("distribution_num_nods.png")
# hist_no_outliers = fig2.get_figure()
# hist_no_outliers.savefig("size_distribution.png")
# hist_all = fig3.get_figure()
# hist_all.savefig("size_distribution_all.png")
# dist_plot_all = fig_4.get_figure()
# dist_plot_all.savefig("size_distribution_all_line_fit.png")

In [123]:
for i in range(len(BMI_data['Severity'])):
    try:
        if BMI_data['Severity'].iloc[i]<BMI_data['Severity'].iloc[i+1]:
            print(i)
    except:
        print('Error in {}'.format(i))
        pass
print('Length of df is {}'.format(len(BMI_data['Severity'])))    
#Above means that we have patients in order, if error in len of dataframe (-1) only

Error in 351
Length of df is 352


### Create size groups 

In [51]:
#For each degree of emphysema print nodule distribution, total number of nodules and confirm that we have the expected num
for degree in new_severity_names:
    print(degree)
    print("Number of nodules and number of patients with that many nodules:{}".format(eval('nodule_num_'+degree)))
    
    #For each number of nodules (0-10) multiply it by th number of occurences and add them all to get total num of nodules
    total=np.sum([a*b for a,b in zip(list(eval('nodule_num_'+degree).keys()),list(eval('nodule_num_'+degree).values()))])
    
    print("Total num of nodules for {} is {}".format(degree,total))
    print('\n')

In [125]:
# print("Low BMI group median size:",np.median(severe_emph_sizes),"IQR:",st.iqr(severe_emph_sizes))
# print("High BMI group median size:",np.median(new_vols_small),"IQR:",st.iqr(new_vols_small))

# res = st.median_test(severe_emph_sizes, new_vols_small)
# print("P value is {}".format(res[1])) #this is pvalue

In [52]:
#Get group sizes - '_100' corresponds to nodules between 30-100mm3 since nodules <30mm3 were not annotated by radiologists
for degree in new_severity_names:
    
    print(degree)
       
    #Create groups based on size of nodules for each degree - eg. sizes_low, sizes_low_30, etc.
    #Each group is: '_30'=nodules smaller than 30mm3, '_30_100' from 30-100mm3 etc.
    exec("sizes_"+degree+"=np.asarray(eval('sizes_'+degree)).astype(int)")
    exec("sizes_"+degree+"_100"+"=sizes_"+degree+"[sizes_"+degree+"<=100]")
    exec("sizes_"+degree+"_100_300"+"=sizes_"+degree+"[(sizes_"+degree+">100) & (sizes_"+degree+"<=300)]")
    exec("sizes_"+degree+"_300"+"=sizes_"+degree+"[(sizes_"+degree+">300)]")
    
    #Create list with indices of the patients of the above groups
    exec("patient_ind_"+degree+"=eval('patient_names_'+degree)") #Needed for patient_names below
    exec("patient_ind_"+degree+"_100=[index for index,value in enumerate("+'sizes_'+degree+") if value <=100]")
    exec("patient_ind_"+degree+"_100_300=[index for index,value in enumerate("+'sizes_'+degree+") if value>100 and value<=300]")
    exec("patient_ind_"+degree+"_300=[index for index,value in enumerate("+'sizes_'+degree+") if value>300]")
    
    #Get the actual names of the above patients for each size group
    exec("patient_"+degree+"_names=patient_names_"+degree)
    exec("patient_"+degree+"_100_names=[patient_ind_"+degree+"[i] for i in patient_ind_"+degree+"_100]")
    exec("patient_"+degree+"_100_300_names=[patient_ind_"+degree+"[i] for i in patient_ind_"+degree+"_100_300]")
    exec("patient_"+degree+"_300_names=[patient_ind_"+degree+"[i] for i in patient_ind_"+degree+"_300]")

    #Number of nodules contained in the particular patient
    exec("patient_nodules_"+degree+"_100=[patient_nodules_"+degree+"[i] for i in patient_ind_"+degree+"_100]")
    exec("patient_nodules_"+degree+"_100_300=[patient_nodules_"+degree+"[i] for i in patient_ind_"+degree+"_100_300]")
    exec("patient_nodules_"+degree+"_300=[patient_nodules_"+degree+"[i] for i in patient_ind_"+degree+"_300]")    
    
    #Confirm that the above worked
    assert len(eval("sizes_"+degree))==len(eval("sizes_"+degree+"_100"))+len(eval("sizes_"+degree+"_100_300"))+len(eval("sizes_"+degree+"_300"))
    assert len(eval("patient_nodules_"+degree))==len(eval("patient_nodules_"+degree+"_100"))+len(eval("patient_nodules_"+degree+"_100_300"))+len(eval("patient_nodules_"+degree+"_300"))
    
    #Print unique patients contained in each degree of BMI - Only those with nodules in the lists below
    #The rest have no nodules in GT annotations
    print(len(np.unique(eval("patient_"+degree+"_names"))))

In [53]:
print("Total num of nodules in high:",len(sizes_high_redcap))
print("Total num of nodules of 30-100mm3 in high:",len(sizes_high_redcap_100))
print("Total num of nodules of 100-300mm3 in high:",len(sizes_high_redcap_100_300))
print("Total num of nodules of 300+mm3 in high:",len(sizes_high_redcap_300))

print("Total num of nodules in low:",len(sizes_low_redcap))
print("Total num of nodules of 30-100mm3 in low:",len(sizes_low_redcap_100))
print("Total num of nodules of 100-300mm3 in low:",len(sizes_low_redcap_100_300))
print("Total num of nodules of 300+mm3 in low:",len(sizes_low_redcap_300))

In [128]:
#Get lists of all participants within each low and high subgroups - even those without nodules
for ind,name in enumerate(new_severity_names):
    print(name)
    exec('all_'+name+'=[]')
    exec("all_"+name+".append(list(BMI_data[BMI_data['BMI']==list(severity_dict.keys())["+str(ind)+"]]['participant_id'].values))")
    exec('all_'+name+'=all_'+name+'[0]')

high_redcap
low_redcap


In [54]:
# np.unique(all_high_redcap) #Check cases with high BMI

In [56]:
# np.unique(all_low_redcap) #Check cases with low BMI

In [57]:
only_300_low=[pat for pat in patient_low_redcap_300_names if pat not in patient_low_redcap_100_300_names
              and pat not in patient_low_redcap_100_names] #These are cases with only nodules >300mm3
print("Total num of participants with low is {}".format(len(all_low_redcap)))
print("Total num of participants with low and with nodules is {}".format(len(list(np.unique(eval("patient_low_redcap_names"))))))
print("Total num of participants with low and without nodules is {}".format(len(list(set(all_low_redcap)-set(list(np.unique(eval("patient_low_redcap_names"))))))))
print("The participants without nodules with low are the following: {}".format(list(set(all_low_redcap)-set(list(np.unique(eval("patient_low_redcap_names")))))))
print('\n')

only_300_high=[pat for pat in patient_high_redcap_300_names if pat not in patient_high_redcap_100_300_names
              and pat not in patient_high_redcap_100_names] #These are cases with only nodules >300mm3
print("Total num of participants with high is {}".format(len(all_high_redcap)))
print("Total num of participants with high and with nodules is {}".format(len(list(np.unique(eval("patient_high_redcap_names"))))))
print("Total num of participants with high and without nodules is {}".format(len(list(set(all_high_redcap)-set(list(np.unique(eval("patient_high_redcap_names"))))))))
print("The participants without nodules with high are the following: {}".format(list(set(all_high_redcap)-set(list(np.unique(eval("patient_high_redcap_names")))))))
print('\n')

In [58]:
#Volumes of nodules of non-emphysema list - Similar as above without participant_id to be used below
only_vols=BMI_data[BMI_data['participant_id'].isin(np.unique(all_pats))].iloc[:,[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

#Get number of nodules of participants in the non-emphysema list
new_vols=[]
for col in only_vols.columns:
    new_vols.append(only_vols[col]) 

new_vols=[i for x in new_vols for i in x]
new_vols=[x for x in new_vols if np.isnan(x)==False]

print("Total num of nodules in high+low BMI list is {}".format(len(new_vols)))
# np.sum(only_vols.count()) #another way to get their number

In [59]:
new_vols_100=[x for x in new_vols if x<=100] #If we want only nodules <=100mm3
new_vols_100_300=[x for x in new_vols if x<=300 and x>100] #If we want only nodules 100<x<=300mm3
new_vols_300=[x for x in new_vols if x>300] #If we want only nodules >300mm3
print("Total num of nodules in high+low BMI list of size 30-100mm3 is {}".format(len(new_vols_100)))
print("Total num of nodules in high+low BMI list of size 100-300mm3 is {}".format(len(new_vols_100_300)))
print("Total num of nodules in high+low BMI list of size >300mm3 is {}".format(len(new_vols_300)))
#We expect to have nodules >300mm3 here, even though not explicitly sampled, since we sample on a scan basis.

In [134]:
end=time.time()
print("Time to execute the script is {} sec".format(end-start)) #~15secs without

Time to execute the script is 5.993586540222168 sec
